# Setup

## Checks

In [ ]:
assert not os.path.exists(DIR_FEATURES), "Features Already Extracted. Proceed only if necessary."

## Imports

In [1]:
import os.path
import bcolz

vai Modules

In [2]:
from vaiutils import path_consts

PyTorch Modules

In [3]:
import torchvision.transforms as transforms
import torch.nn as nn

from torch.utils.data.dataloader import DataLoader
from torchvision.datasets import ImageFolder
from torchvision.models import vgg16
from torch.autograd import Variable

## Define useful variables and functions

In [4]:
for k, v in path_consts('COCO'):
    exec(k + '=v')

In [8]:
def save_tensor(filename, tensor):
    bcolz.carray(tensor, rootdir=filename, mode='w').flush()

## Load Data

In [20]:
batch_size = 32

In [21]:
transform = transforms.Compose([transforms.Scale(224),
                                transforms.CenterCrop(224),
                                transforms.ToTensor(),
                                transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])
data = DataLoader(ImageFolder(DIR_DATA, transform))

# Extract Image Features using CNN

In [23]:
DIR_FEATURES = os.path.join(DIR_CHECKPOINTS, 'features')

In [24]:
vgg = vgg16(pretrained=True).cuda()
vgg.classifier = nn.Sequential(*list(vgg.classifier.children())[:-2])

In [25]:
vgg.eval()
print("Total VGG parameters: {:,}".format(sum([prod(p.data.shape) for p in vgg.parameters()])))

Total VGG parameters: 134,260,544


In [26]:
os.makedirs(DIR_FEATURES, exists_ok=True)
cnn_features = np.zeros((len(data), 4096))
data_iterator = iter(data)

for i in tqdm_notebook(range(len(cnn_features))):
    cnn_features[i] = vgg(Variable(data_iterator.next()[0].cuda(), volatile=True)).data.cpu().numpy()
save_tensor(DIR_FEATURES, cnn_features)